# Tugas Besar Pemroseran Parallel Kelas B
## Analisis Data Games menggunakan MPI (reduce, filter), pyspark context (reduce, filter), dan sparksession (query) 

1. Rakasya Yoga Surya Prastama (F1D02310022)
2. Rakasya Yoga Surya Prastama (F1D02310022)
3. Rakasya Yoga Surya Prastama (F1D02310022)
4. Rakasya Yoga Surya Prastama (F1D02310022)
5. Rakasya Yoga Surya Prastama (F1D02310022)

Link Kaggle untuk dataset: https://www.kaggle.com/datasets/jummyegg/rawg-game-dataset

In [21]:
%pip install pyspark
%pip install mpi4py

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
from mpi4py import MPI
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import csv

In [23]:
import time
mulai_ukur = time.time()
akhir_ukur = time.time()

waktu_pengerjaan = akhir_ukur - mulai_ukur

print(waktu_pengerjaan)

0.0


In [32]:
spark = SparkSession.builder.appName("Game Sales Analysis").getOrCreate()
data = spark.read.csv("game_info.csv", header=True, inferSchema=True)
data.show(10)

+------+--------------------+--------------------+----------+----------+-----+-------------------+--------------------+------+----------+--------+------------------+-------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+
|    id|                slug|                name|metacritic|  released|  tba|            updated|             website|rating|rating_top|playtime|achievements_count|ratings_count|suggestions_count|game_series_count|reviews_count|           platforms|          developers|              genres|          publishers| esrb_rating|added_status_yet|added_status_owned|added_status_beaten|added_status_toplay|added_status_dropped|added_status_playing|
+------+--------------------+--------------------+----------+----------+-----+-------------------+------------

In [41]:
mpi = MPI.COMM_WORLD
size = mpi.Get_size()

df = pd.read_csv("game_info.csv")
chunks = [df.iloc[i::size].reset_index(drop=True) for i in range(size)]

# Broadcast potongan data
scatter = mpi.scatter(chunks, root=0)

# FILTER: metacritic > 80
Filter = scatter[scatter["metacritic"] > 80]

# REDUCE: jumlah ratings_count
Reduce = Filter["ratings_count"].sum()
total_Reduce = mpi.reduce(Reduce, op=MPI.SUM, root=0)

print("Total ratings_count untuk metacritic > 80:", total_Reduce)


Total ratings_count untuk metacritic > 80: 427925


In [ ]:
# Buat SparkSession dan SparkContext
sc = spark.sparkContext

# Baca file CSV sebagai RDD
rdd = sc.textFile("game_info.csv")

# Ambil header
header = rdd.first()

# Hilangkan header dan parsing CSV
data = rdd.filter(lambda x: x != header).map(lambda row: next(csv.reader([row])))

# FILTER: metacritic > 80 (kolom ke-4, index 3)
filtered = data.filter(lambda x: len(x) > 3 and x[3].isdigit() and int(x[3]) > 80)

# REDUCE: total ratings_count (kolom ke-13, index 12 — sesuaikan jika struktur berubah)
ratings_count_index = 12
ratings = filtered.map(lambda x: int(x[ratings_count_index]) if len(x) > ratings_count_index and x[ratings_count_index].isdigit() else 0)

# Totalkan dengan reduce
total = ratings.reduce(lambda a, b: a + b)

# Tampilkan hasil
print("Total ratings_count (PySpark RDD):", total)


Total ratings_count (PySpark RDD): 427925


In [40]:
# data = spark.read.csv("game_info.csv", header=True, inferSchema=True)
data.createOrReplaceTempView("games")

result = spark.sql("""
    SELECT name, rating, metacritic, platforms, released, genres, publishers 
    FROM games
    WHERE 
        rating IS NOT NULL AND 
        metacritic IS NOT NULL AND
        TRY_CAST(rating AS FLOAT) IS NOT NULL AND 
        metacritic IS NOT NULL
    ORDER BY rating  DESC
    LIMIT 20
""")

print("Hasil Query:")
result.toPandas()

Hasil Query:


,name,rating,metacritic,platforms,released,genres,publishers
0,Persona 5 Royal,4.81,95,PlayStation 4,2020-03-31,Adventure||RPG,SEGA||Atlus
1,The Witcher 3: Game of the Year,4.77,92,Xbox One||PlayStation 4||Nintendo Switch||PC,2016-08-30,Adventure||RPG,CD PROJEKT RED
2,The Witcher 3: Wild Hunt - Blood and Wine,4.75,92,PC||Xbox One||PlayStation 4,2016-05-30,RPG,CD PROJEKT RED
3,The Witcher 3: Wild Hunt - Hearts of Stone,4.74,90,PC||Xbox One||PlayStation 4,2015-10-13,RPG,CD PROJEKT RED||cdp.pl
4,Yakuza: Like a Dragon,4.73,84,PlayStation 5||Xbox Series S/X||Xbox One||PC||...,2020-01-16,Action||RPG,SEGA
5,Grand Theft Auto IV: Complete Edition,4.73,95,Xbox 360||PlayStation 3||PC,2010-10-27,Action||Adventure,Rockstar Games
6,The Last Of Us Remastered,4.7,95,PlayStation 4,2014-07-29,Action||Adventure,Sony Computer Entertainment
7,Dark Souls III: The Ringed City,4.7,83,PlayStation 4||Xbox One||PC,2017-03-28,Action||RPG,Bandai Namco Entertainment
8,Drill Dozer (2005),4.67,81,Game Boy Advance||Wii U,2005-09-22,Action,Nintendo
9,Kingdom Hearts II Final Mix,4.67,87,PlayStation 2||PlayStation 3||PlayStation 4,2007-03-29,Action||RPG,Square Enix
